In [1]:
# imports 
import numpy as np
from math import sqrt 
from pprint import pprint
from numpy import array
from scipy.io import arff
import pandas as pd

In [2]:
# load data

df=pd.read_csv('Cardiotocography_02_v10.csv')  

del df['id']
del df['Unnamed: 0']
df['outlier'] = df.outlier.apply(lambda label: 1 if label == "'yes'" else 0)

df.head()

,LB,AC,FM,UC,ASTV,MSTV,ALTV,MLTV,DL,DS,...,Min,Max,Nmax,Nzeros,Mode,Mean,Median,Variance,Tendency,outlier
0,146.0,0.0,0.0,5.0,65.0,0.4,33.0,7.4,0.0,0.0,...,134.0,164.0,1.0,0.0,150.0,149.0,151.0,1.0,0.0,1
1,128.0,0.0,0.0,2.0,86.0,0.3,79.0,2.9,0.0,0.0,...,114.0,130.0,0.0,0.0,128.0,126.0,129.0,0.0,1.0,1
2,149.0,0.0,0.0,5.0,61.0,0.4,34.0,5.6,0.0,0.0,...,148.0,160.0,1.0,0.0,154.0,153.0,155.0,0.0,0.0,1
3,122.0,0.0,0.0,0.0,83.0,0.5,6.0,15.6,0.0,0.0,...,62.0,130.0,0.0,0.0,122.0,122.0,123.0,3.0,1.0,1
4,134.0,0.0,4.0,0.0,79.0,0.2,42.0,5.5,0.0,0.0,...,128.0,145.0,2.0,0.0,135.0,135.0,136.0,1.0,0.0,1


In [3]:
df.shape

(1688, 22)

In [4]:
# data to values
df = df.values

In [5]:
# Extratrees Deep
import argparse
import sys
import pickle
from sklearn.metrics import accuracy_score
sys.path.insert(0, "lib")
from gcforest.gcforest import GCForest
from gcforest.utils.config_utils import load_json
config = load_json("./examples/Cardiotocography.json")
gc = GCForest(config)

In [7]:
# train test 
from sklearn.cross_validation import train_test_split
y = df[:,21]
X = df[:,0:21]

X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [7]:
# number of class
len(np.unique(y_train))

2

In [8]:
         # X_enc is the concatenated predict_proba result of ExtraTrees

X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)
   

[ 2018-04-25 00:27:13,582][cascade_classifier.fit_transform] X_groups_train.shape=[(1130, 21)],y_train.shape=(1130,),X_groups_test.shape=[(558, 21)],y_test.shape=(558,)
[ 2018-04-25 00:27:13,584][cascade_classifier.fit_transform] group_dims=[21]
[ 2018-04-25 00:27:13,586][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-04-25 00:27:13,588][cascade_classifier.fit_transform] group_ends=[21]
[ 2018-04-25 00:27:13,589][cascade_classifier.fit_transform] X_train.shape=(1130, 21),X_test.shape=(558, 21)
[ 2018-04-25 00:27:13,591][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(1130, 21), X_cur_test.shape=(558, 21)
[ 2018-04-25 00:27:14,271][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=97.37%
[ 2018-04-25 00:27:15,123][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=97.37%
[ 2018-04-25 00:27:15,892][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_

In [9]:
# predict
y_pred = gc.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print("Test Accuracy of ExtraTrees = {:.6f} %".format(acc * 100))

[ 2018-04-25 00:27:21,654][cascade_classifier.transform] X_groups_test.shape=[(558, 21)]
[ 2018-04-25 00:27:21,656][cascade_classifier.transform] group_dims=[21]
[ 2018-04-25 00:27:21,659][cascade_classifier.transform] X_test.shape=(558, 21)
[ 2018-04-25 00:27:21,661][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(558, 21)


Test Accuracy of ExtraTrees = 98.566308 %


In [10]:
# Make data + estimators
X_test_enc = gc.transform(X_test)
X_train_enc = X_train_enc.reshape((X_train_enc.shape[0], -1))
X_test_enc = X_test_enc.reshape((X_test_enc.shape[0], -1))
X_train_origin = X_train.reshape((X_train.shape[0], -1))
X_test_origin = X_test.reshape((X_test.shape[0], -1))
X_train_enc = np.hstack((X_train_origin, X_train_enc))
X_test_enc = np.hstack((X_test_origin, X_test_enc))

[ 2018-04-25 00:27:23,149][cascade_classifier.transform] X_groups_test.shape=[(558, 21)]
[ 2018-04-25 00:27:23,151][cascade_classifier.transform] group_dims=[21]
[ 2018-04-25 00:27:23,152][cascade_classifier.transform] X_test.shape=(558, 21)
[ 2018-04-25 00:27:23,155][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(558, 21)


In [11]:
# New data
X_train=X_train_enc
X_test=X_test_enc 
X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)

[ 2018-04-25 00:27:24,776][cascade_classifier.fit_transform] X_groups_train.shape=[(1130, 23)],y_train.shape=(1130,),X_groups_test.shape=[(558, 23)],y_test.shape=(558,)
[ 2018-04-25 00:27:24,779][cascade_classifier.fit_transform] group_dims=[23]
[ 2018-04-25 00:27:24,780][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-04-25 00:27:24,782][cascade_classifier.fit_transform] group_ends=[23]
[ 2018-04-25 00:27:24,784][cascade_classifier.fit_transform] X_train.shape=(1130, 23),X_test.shape=(558, 23)
[ 2018-04-25 00:27:24,787][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(1130, 23), X_cur_test.shape=(558, 23)
[ 2018-04-25 00:27:25,448][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=98.25%
[ 2018-04-25 00:27:26,205][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=97.37%
[ 2018-04-25 00:27:27,013][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_

In [12]:
# predict  data + Es
y_pred = gc.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print("Test Accuracy of ExtraTrees = {:.6f} %".format(acc * 100))

[ 2018-04-25 00:27:32,945][cascade_classifier.transform] X_groups_test.shape=[(558, 23)]
[ 2018-04-25 00:27:32,947][cascade_classifier.transform] group_dims=[23]
[ 2018-04-25 00:27:32,950][cascade_classifier.transform] X_test.shape=(558, 23)
[ 2018-04-25 00:27:32,952][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(558, 23)


Test Accuracy of ExtraTrees = 99.103943 %


In [13]:
def update(X_train,X_test):
    # update Estimators
    M = np.delete(X_train, (-1), axis=1)
    X_train= np.delete(M, (-1), axis=1)
    M = np.delete(X_test, (-1), axis=1)
    X_test = np.delete(M, (-1), axis=1)
    return X_train,X_test

def newdata(X_test_enc, X_train_enc):
    # Make Data + Estimators
    X_train_origin = X_train.reshape((X_train.shape[0], -1))
    X_test_origin = X_test.reshape((X_test.shape[0], -1))
    X_train_enc = np.hstack((X_train_origin, X_train_enc))
    X_test_enc = np.hstack((X_test_origin, X_test_enc))
    return X_test_enc, X_train_enc

def affiche():
    # predict
    y_pred = gc.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print("Test Accuracy of ExtraTrees = {:.6f} %".format(acc * 100),i,'fois')

In [14]:
# Deep
for i in range (20):
    X_train,X_test=update(X_train,X_test)
    X_test_enc, X_train_enc=newdata(X_test_enc, X_train_enc)

    X_train=X_train_enc
    X_test=X_test_enc 
    X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)
    affiche()

[ 2018-04-25 00:27:34,462][cascade_classifier.fit_transform] X_groups_train.shape=[(1130, 23)],y_train.shape=(1130,),X_groups_test.shape=[(558, 23)],y_test.shape=(558,)
[ 2018-04-25 00:27:34,464][cascade_classifier.fit_transform] group_dims=[23]
[ 2018-04-25 00:27:34,465][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-04-25 00:27:34,467][cascade_classifier.fit_transform] group_ends=[23]
[ 2018-04-25 00:27:34,469][cascade_classifier.fit_transform] X_train.shape=(1130, 23),X_test.shape=(558, 23)
[ 2018-04-25 00:27:34,471][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(1130, 23), X_cur_test.shape=(558, 23)
[ 2018-04-25 00:27:35,100][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=98.25%
[ 2018-04-25 00:27:36,039][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=95.61%
[ 2018-04-25 00:27:36,856][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_

('Test Accuracy of ExtraTrees = 98.924731 %', 0, 'fois')


[ 2018-04-25 00:27:44,942][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=98.25%
[ 2018-04-25 00:27:45,751][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=95.61%
[ 2018-04-25 00:27:46,579][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=94.74%
[ 2018-04-25 00:27:47,420][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=98.25%
[ 2018-04-25 00:27:48,228][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=98.23%
[ 2018-04-25 00:27:49,005][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=97.35%
[ 2018-04-25 00:27:49,838][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=99.11%
[ 2018-04-25 00:27:50,647][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=98.21%


('Test Accuracy of ExtraTrees = 99.283154 %', 1, 'fois')


[ 2018-04-25 00:27:54,612][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=98.25%
[ 2018-04-25 00:27:55,402][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=94.74%
[ 2018-04-25 00:27:56,217][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=94.74%
[ 2018-04-25 00:27:57,034][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=98.25%
[ 2018-04-25 00:27:57,867][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=98.23%
[ 2018-04-25 00:27:58,686][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=97.35%
[ 2018-04-25 00:27:59,620][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=99.11%
[ 2018-04-25 00:28:00,382][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=97.32%


...

('Test Accuracy of ExtraTrees = 98.924731 %', 17, 'fois')


[ 2018-04-25 00:30:29,962][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=97.37%
[ 2018-04-25 00:30:30,756][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=92.98%
[ 2018-04-25 00:30:31,548][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=98.25%
[ 2018-04-25 00:30:32,362][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=97.37%
[ 2018-04-25 00:30:33,172][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=98.23%
[ 2018-04-25 00:30:34,009][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=96.46%
[ 2018-04-25 00:30:34,796][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=98.21%
[ 2018-04-25 00:30:35,593][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=98.21%


('Test Accuracy of ExtraTrees = 98.924731 %', 18, 'fois')


[ 2018-04-25 00:30:39,502][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=97.37%
[ 2018-04-25 00:30:40,307][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=92.98%
[ 2018-04-25 00:30:41,140][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=97.37%
[ 2018-04-25 00:30:41,969][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=97.37%
[ 2018-04-25 00:30:42,768][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=98.23%
[ 2018-04-25 00:30:43,567][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=97.35%
[ 2018-04-25 00:30:44,356][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=98.21%
[ 2018-04-25 00:30:45,154][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=98.21%


('Test Accuracy of ExtraTrees = 98.566308 %', 19, 'fois')


# ('Test Accuracy of ExtraTrees = 99.283154 %', 1, 'fois')